### LeNEt
Source(CN):https://zhuanlan.zhihu.com/p/28333410   
Code: https://github.com/BIGBALLON/cifar-10-cnn/tree/master/1_Lecun_Network  


Source(CN): https://zhuanlan.zhihu.com/p/22447440

![LeNet](Figure/LeNet.jpg)

### Subsampling = Pooling
下采样 = 池化

### Some Tricks
Source(EN): http://lamda.nju.edu.cn/weixs/project/CNNTricks/CNNTricks.html

### Batch_Size vs. interations vs. Epochs
Source(EN): https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9

### 1. LeNet

In [15]:
import keras
from keras import optimizers
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.regularizers import l2

In [18]:
batch_size    = 128
epochs        = 10
iterations    = 391
num_classes   = 10
log_filepath  = './lenet'
weight_decay  = 0.0001

In [16]:
# add weight decay
def build_model():
    model = Sequential()
    model.add(Conv2D(6,(5,5),padding='valid',activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(weight_decay),input_shape=(32,32,3)))
    model.add(MaxPooling2D((2,2),strides=(2,2)))
    model.add(Conv2D(16,(5,5),padding='valid',activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(weight_decay)))
    model.add(MaxPooling2D((2,2),strides=(2,2)))
    model.add(Flatten())
    model.add(Dense(120,activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(weight_decay)))
    model.add(Dense(84,activation='relu',kernel_initializer='he_normal',kernel_regularizer=l2(weight_decay)))
    model.add(Dense(10,activation='softmax',kernel_initializer='he_normal',kernel_regularizer=l2(weight_decay)))
    sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])
    return model


In [4]:
def scheduler(epoch):
    learning_rate_init = 0.02
    if epoch >= 80:
        learning_rate_init = 0.01
    if epoch >= 150:
        learning_rate_init = 0.004
    return learning_rate_init

In [5]:
# load data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [6]:
x_train.shape

(50000, 32, 32, 3)

In [7]:
# Normalization
mean  = [125.307, 122.95, 113.865]
std   = [62.9932, 62.0887, 66.7048]
for i in range(3):
    x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
    x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]


In [19]:
# build network
model = build_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 6)         456       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 120)               48120     
_________________________________________________________________
dense_5 (Dense)              (None, 84)                10164     
__________

In [9]:
 # set callback
tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
cbks = [change_lr,tb_cb]    

In [20]:
# data augmentation
from keras.preprocessing.image import ImageDataGenerator
print('Using real-time data augmentation.')
datagen = ImageDataGenerator(horizontal_flip=True,
            width_shift_range=0.125,height_shift_range=0.125,fill_mode='constant',cval=0.)

datagen.fit(x_train)

    # start train 
model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                        steps_per_epoch=iterations,
                        epochs=epochs,
                        callbacks=cbks,
                        validation_data=(x_test, y_test))

Using real-time data augmentation.
Epoch 1/10
391/391 [==============================] - 54s 138ms/step - loss: 1.9419 - acc: 0.2982 - val_loss: 1.6254 - val_acc: 0.4224
Epoch 2/10
391/391 [==============================] - 48s 124ms/step - loss: 1.6513 - acc: 0.4158 - val_loss: 1.5016 - val_acc: 0.4804
Epoch 3/10
391/391 [==============================] - 49s 124ms/step - loss: 1.5409 - acc: 0.4578 - val_loss: 1.4496 - val_acc: 0.4989
Epoch 4/10
391/391 [==============================] - 48s 124ms/step - loss: 1.4652 - acc: 0.4876 - val_loss: 1.3701 - val_acc: 0.5301
Epoch 5/10
391/391 [==============================] - 49s 125ms/step - loss: 1.4166 - acc: 0.5084 - val_loss: 1.3840 - val_acc: 0.5372
Epoch 6/10
391/391 [==============================] - 48s 124ms/step - loss: 1.3886 - acc: 0.5214 - val_loss: 1.3091 - val_acc: 0.5560
Epoch 7/10
391/391 [==============================] - 51s 129ms/step - loss: 1.3550 - acc: 0.5363 - val_loss: 1.3793 - val_acc: 0.5468
Epoch 8/10
391/391 [

In [10]:
# start traing 
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,callbacks=cbks,validation_data=(x_test, y_test), shuffle=True)
    # save model
#model.save('lenet.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 43s 854us/step - loss: 1.7778 - acc: 0.3520 - val_loss: 1.5074 - val_acc: 0.4409
Epoch 2/10
50000/50000 [==============================] - 42s 840us/step - loss: 1.4654 - acc: 0.4689 - val_loss: 1.4448 - val_acc: 0.4743
Epoch 3/10
50000/50000 [==============================] - 46s 917us/step - loss: 1.3483 - acc: 0.5161 - val_loss: 1.4604 - val_acc: 0.4814
Epoch 4/10
50000/50000 [==============================] - 46s 922us/step - loss: 1.2787 - acc: 0.5438 - val_loss: 1.2917 - val_acc: 0.5381
Epoch 5/10
50000/50000 [==============================] - 45s 895us/step - loss: 1.2219 - acc: 0.5658 - val_loss: 1.3117 - val_acc: 0.5474
Epoch 6/10
50000/50000 [==============================] - 44s 879us/step - loss: 1.1690 - acc: 0.5818 - val_loss: 1.2256 - val_acc: 0.5762
Epoch 7/10
50000/50000 [==============================] - 47s 945us/step - loss: 1.1259 - acc: 0.5972 - val_loss: 1.

### 2. LeNet improvement (10 Epochs)
0. BaseModel acc(0.6486)
1. nomalization  acc(0.5962)
2. data augmentation acc(0.6299)
3. weight decay (0.5917)   